### Import libraries

In [1]:
# Import libraries for the query
import time
import hmac
from requests import Request, Session, Response
import requests

# Import libraries for model loading
import tensorflow as tf
import sklearn as skl

# Import generally useful libraries
import numpy as np
import pandas as pd
import matplotlib as mpl

# Import custom libraries to compute indicators
from package import euklid_regressor as eur 
from package import indicator as ind

# Import custom library to interface with FTX API
from package import FTXClient
import datetime as datetime

# Load environemnt variables
from os import getenv
from pathlib import Path
from dotenv import load_dotenv
dotenv_path = Path('Trading Bot/Bot/.env')
load_dotenv(dotenv_path=dotenv_path)

# Set off warnings
import warnings
import pytz
warnings.filterwarnings("ignore")


In [3]:
pytz.timezone("Europe/London")
today = datetime.datetime.now(pytz.timezone("Europe/Rome"))
if today.day == datetime.datetime.now(pytz.timezone("Europe/Rome")).day:
    print("Yes")
else:
    print("No")

Yes


### Import LSTM model

In [ ]:
path = 'C://users/valer/LSTM_saved_model' # Insert path where the model has been saved
model = tf.keras.models.load_model(path)

### Sample code for computational time checking

In [4]:
# Save code to check execution time
# start_time = time.time()
# print('Time elapsed: ' + str(time.time()-start_time))

# Initiate TCP Session
s = requests.Session()

# Set subaccount access credentials
dotenv_path = Path('Trading Bot/Bot/.env')
api_key = getenv('API_KEY')
api_secret = getenv('API_SECRET')
subaccount_name = getenv('SUBACCOUNT_NAME')

# Initialize useful variables
CLIENT_ID = 0
ID = 0
ORDER_ID = 0

In [5]:
# Create client instance to use the methods
client = FTXClient.FtxClient(api_key, api_secret,subaccount_name)
from datetime import date
today = datetime.datetime.now(pytz.timezone("Europe/Rome"))
print(today)

2022-08-01 15:42:03.201964+02:00


### Strategy execution

There are two types of execution: market and limit. The former is the easiest one and can be used when the
trading balance isn't large enough to justify a more efficient order execution engine. The latter consists of 
a series of limit orders placed right underneeth the current market price. It should take at most 20 minutes to execute limit
orders (for both closing the current position and opening a new one). If the time passes without the full position size
being filled it executes a market order to fill the remaining size.

#### Market execution

In [ ]:
while True:
    # REMEMBER TO CHANGE BACK TO TODAY.DAY!!!!!!
    if today.minute == datetime.datetime.now(pytz.timezone("Europe/Rome")).minute:
        pass
    else:
        today = datetime.datetime.now(pytz.timezone("Europe/Rome"))
        end_time = int(round(datetime.datetime.now(pytz.timezone("Europe/Rome")).timestamp()))
        price = client.get_historical_prices(market = 'BTC-PERP', start_time = 1649541600, end_time = end_time)
        orders_placed = client.get_orders_placed()
        positions = client.get_positions()
        balances = client.get_balances()[0]['total']
        # Reconstruct the data to pass into the model
        close = [i['close'] for i in price]
        print(close[-1])
        df2 = pd.DataFrame({'Close': close})
        eur.macd(df2,df2['Close'])
        df2['SO'] = eur.Stochastic_Oscillator(df2['Close'])
        df2['1 Day ROI']=ind.ROI(df2,1)
        df2['2 Day ROI']=ind.ROI(df2,2)
        df2['3 Day ROI']=ind.ROI(df2,3)
        df2['RSI_1'] = eur.rsi(df2['Close'],1)
        df2['RSI_2'] = eur.rsi(df2['Close'],2)
        df2['RSI_3'] = eur.rsi(df2['Close'],3)
        df2['RSI_14'] = eur.rsi(df2['Close'],14)
        differenced = (df2['Close'] - df2['Close'].shift(1))/(df2['Close'].shift(1))[1:]
        previous_differenced = differenced.shift(1)
        df2["Differenced"] = differenced
        df2["Previous_differenced"] = previous_differenced
        x = df2[['SO','RSI_1', 'RSI_2', 'RSI_3', 'RSI_14', '1 Day ROI', '2 Day ROI', '3 Day ROI', 'MACD_12_26_9', 'MACDh_12_26_9',
          'MACDs_12_26_9', "Previous_differenced"]]
        x = x.iloc[33:,:]
        X = x.values.reshape((x.shape[0], 1, x.shape[1]))
        X = X.astype('float32')
        X = tf.convert_to_tensor(X)
        predict = model.predict(X)
        if predict[-1] >= 0 and predict[-2] >= 0:
            if not positions is True:
                # Send POST request to place buy order
                print('Send buy order on: ', today)
                balance = client.get_balances()[0]['total']
                size = balance / close[-1]
                CLIENT_ID += 1
                client.place_order('BTC-PERP', 'buy', price=None, size=size, client_id=str(CLIENT_ID), type='market')
                # ORDER_ID = client.get_orders_placed()[0]['id']
            else:
                print('Keep long position on: ', today)
        elif predict[-1] >= 0 and predict[-2] <= 0:
            if not positions is True:
                # Send POST request to place sell order
                print('Send buy order on: ', today)
                balance = client.get_balances()[0]['total']
                size = balance / close[-1]
                CLIENT_ID += 1
                client.place_order('BTC-PERP', 'buy', price=None, size=size, client_id=str(CLIENT_ID), type='market')
                # ORDER_ID = client.get_orders_placed()[0]['id']
            else:
                print('Close short and go long on: ', today)
                CLIENT_ID += 1
                size = client.get_positions()['size']
                client.place_order('BTC-PERP', 'buy', price=None, size=size, client_id=str(CLIENT_ID), type='market',
                                   reduce_only=True)
                balance = client.get_balances()[0]['total']
                size = balance / close[-1]
                CLIENT_ID += 1
                client.place_order('BTC-PERP', 'buy', price=None, size=size, client_id=str(CLIENT_ID), type='market')
                # ORDER_ID = client.get_orders_placed()[0]['id']

        elif predict[-1] < 0 and predict[-2] >= 0:
            if not positions is True:
                # Send POST request to place buy order
                print('Send sell order on: ', today)
                balance = client.get_balances()[0]['total']
                size = balance / close[-1]
                CLIENT_ID += 1
                client.place_order('BTC-PERP', 'sell', price=None, size=size, client_id=str(CLIENT_ID), type='market')
                # ORDER_ID = client.get_orders_placed()[0]['id']
            else:
                print('Close long and go short on: ', today)
                CLIENT_ID += 1
                size = client.get_positions()['size']
                client.place_order('BTC-PERP', 'sell', price=None, size=size, client_id=str(CLIENT_ID), type='market',
                                   reduce_only=True)
                balance = client.get_balances()[0]['total']
                size = balance / close[-1]
                CLIENT_ID += 1
                client.place_order('BTC-PERP', 'sell', price=None, size=size, client_id=str(CLIENT_ID), type='market')
                # ORDER_ID = client.get_orders_placed()[0]['id']
        else:
            if not positions is True:
                # Send POST request to place buy order
                print('Send sell order on: ', today)
                balance = client.get_balances()[0]['total']
                size = balance / close[-1]
                CLIENT_ID += 1
                client.place_order('BTC-PERP', 'sell', price=None, size=size, client_id=str(CLIENT_ID), type='market')
                # ORDER_ID = client.get_orders_placed()[0]['id']
            else:
                print('Keep short position on: ', today)

In [14]:
positions = client.get_positions()
balance = client.get_balances()[0]['total']
futures_list = client.get_positions()['size']
print((futures_list, balance))
time.sleep(4)
print('----------------------\n', positions['size'] == 0.0)

https://ftx.com/api/positions
https://ftx.com/api/wallet/balances
https://ftx.com/api/positions
(0.0, 109.58614543)
----------------------
 True
